In [17]:
import os

def delete_result_statistics_logs(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if 'result_statistics' in file:
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f'Deleted: {file_path}')

if __name__ == "__main__":
    log_dir = 'logs'  # 指定你的目录
    delete_result_statistics_logs(log_dir)

Deleted: logs/plan_attack/llama3.1:70b/result_statistics.log
Deleted: logs/plan_attack/gpt-4o-mini/result_statistics.log
Deleted: logs/clean/qwen2:72b/result_statistics_.log
Deleted: logs/clean/llama3:8b/result_statistics_.log
Deleted: logs/clean/llama3.1:70b/result_statistics_.log
Deleted: logs/clean/gpt-4o-2024-08-06/result_statistics_.log
Deleted: logs/clean/llama3:70b/result_statistics_.log
Deleted: logs/clean/llama3.1:8b/result_statistics_.log
Deleted: logs/clean/gpt-3.5-turbo/result_statistics_.log
Deleted: logs/clean/mixtral:8x7b/result_statistics_.log
Deleted: logs/clean/gemma2:27b/result_statistics_.log
Deleted: logs/clean/previous/Meta-Llama-3.1-70B-Instruct/result_statistics.log
Deleted: logs/clean/previous/llama3.1:8b/result_statistics.log
Deleted: logs/clean/previous/llama3.1:70b/result_statistics.log
Deleted: logs/clean/previous/llama3:8b/result_statistics.log
Deleted: logs/clean/previous/qwen2:72b/result_statistics.log
Deleted: logs/clean/previous/gemma2:27b/result_stati